<a href="https://colab.research.google.com/github/Umberto28/Deep_Learning_Project/blob/main/Feature_estraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.calibration import LabelEncoder
import tensorflow as tf
import keras
from keras.models import Model
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.efficientnet import preprocess_input

#1 Creazione dataset

carichiamo i valori corretti di ogni diagnosi per pazione in un dataframe

In [ ]:
users = pd.read_csv('/content/users.txt', sep = '\t', dtype = str)

users

,ID,diag,sex,hand,age
0,00006,DYSGR,F,R,15
1,00007,DYSGR,M,R,15
2,00008,DYSGR,M,R,14
3,00011,DYSGR,M,R,8
4,00013,DYSGR,M,R,14
...,...,...,...,...,...
115,00187,0,M,R,15
116,00189,0,M,L,15
117,00190,0,F,R,15
118,00191,0,F,R,15


In [ ]:
users['diag'] = LabelEncoder().fit_transform(users.diag.values)
users=users.drop(columns=["sex","hand","age"])
users

,ID,diag
0,00006,1
1,00007,1
2,00008,1
3,00011,1
4,00013,1
...,...,...
115,00187,0
116,00189,0
117,00190,0
118,00191,0


assegnamo ad ogni paziente l'immagine corrispondende

In [ ]:
def featureExtract(model, users):
  features={}
  features=pd.DataFrame(features)
  for id in users["ID"]:
    img_path = "/content/user"+id+".png"
    img = keras.utils.load_img(img_path, target_size=(299, 299))
    x = keras.utils.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature=model.predict(x)[0]
    row=pd.DataFrame([feature])
    features=pd.concat([features,row], ignore_index=True)
  return features


In [ ]:

modelX=tf.keras.applications.xception.Xception(weights='imagenet',include_top=True)
modelX=Model(inputs=modelX.inputs, outputs=modelX.layers[-2].output)


features=featureExtract(modelX, users)

features['diag']=users['diag']

features.to_csv('/content/xceptionFeatures.csv')


1/1 [==============================] - 1s 526ms/step


In [56]:

modelE=tf.keras.applications.EfficientNetB4(weights='imagenet',include_top=True)
modelE=Model(inputs=modelX.inputs, outputs=modelX.layers[-1].output)

features=featureExtract(modelE, users)

features['diag']=users['diag']

features.to_csv('/content/EfficientNetB4Features.csv')


1/1 [==============================] - 0s 343ms/step
